In [2]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
from datetime import date, timedelta
from dotenv import load_dotenv
from selenium.webdriver.common.keys import Keys
import os
import time

# carregando as variáveis de ambiente
load_dotenv()


True

In [3]:
def retornar_periodo():
    hoje = date.today()
    amanha = hoje + timedelta(days=1)
    hoje_str = date.strftime(hoje, '%d/%m/%Y')
    amanha_str = date.strftime(amanha, '%d/%m/%Y')
    periodo_str = (f'{hoje_str} - {amanha_str}')
    return periodo_str


In [4]:
def obter_driver():
    try:
        # obtendo o usuário logado
        usuario = os.getlogin()
        chrome_options = Options()
        chrome_options.add_argument(
            f'user-data-dir=C:/Users/{usuario}/AppData/Local/Google/Chrome/Selenium'
        )
        service = Service(ChromeDriverManager().install())
        driver = webdriver.Chrome(service=service, options=chrome_options)

        # maximixando a tela
        driver.maximize_window()
        return driver
    except Exception as e:
        raise RuntimeError(f"Erro ao obter o driver: {e}")

In [5]:
def logar(driver):
    try:
        # indo para a tela de login
        driver.get("https://qualylab.gerencialab.com.br/")

        # aguardando elemento aparecer na tela
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.XPATH, '//img[@src="/assets/images/Logo_Gerencialab-azul.png"]'))
            )
        time.sleep(4)

        # obtendo as variáveis de ambiente
        login = os.getenv('LOGIN')
        password = os.getenv('PASSWORD')

        if not login or not password:
            raise ValueError('Variáveis de ambiente LOGIN ou PASSWORD não definidas.')

        # preenchendo o campo usuário
        driver.find_element(By.XPATH, '//*[@id="loginsite"]').send_keys(login)

        # preenchendo o campo senha
        driver.find_element(By.XPATH, '//*[@id="senhasite"]').send_keys(password)

        # clicando em acessar
        driver.find_element(By.XPATH, '//*[@id="authLogin"]').click()

        # aguardando logo GerenciaLab Aparecer na tela
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.XPATH, '//img[@src="/assets/images/gerencialab-logo-n.png"]'))
            )
        time.sleep(4)

        print('Login realizado com sucesso!')
    except Exception as e:
        raise RuntimeError(f'Erro ao realizar login no sistema: {e}')

In [6]:
# desenvolvendo uma função que aplique os filtros
def aplicar_configuracoes(driver):
    colunas = ['Ordem Serviço', 'Status O.S', 'Referência', 'Prioridade', 'Cliente', 'Data de Entrega']
    try:
        # indo para ordens de serviço
        driver.get("https://qualylab.gerencialab.com.br/service-order")
        # aguardando label Ordem serviços aparecer na tela
        WebDriverWait(driver, 30).until(
            EC.presence_of_element_located((By.XPATH, '//h4[text()="Ordem serviços"]'))
            )
        time.sleep(4)
        # clicando no botão Limpar Pesquisa
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//span[text()="Limpar Pesquisa"]'))
            ).click()
        time.sleep(3)
        # clicando em visualizar colunas
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//button[@aria-controls="tableOrdemdeServico"]//span[text()="Visualizar Colunas"]'))
            ).click()
        elementos = driver.find_elements(By.XPATH, '//div[@class="dt-button-collection dropdown-menu"]//a')
        # indo até a coluna Data de Entrega - posição 15 
        for index, elemento in enumerate(elementos):
            if index > 15:
                break; 
            coluna = elemento.text
            status = elemento.get_attribute('class')
            if coluna in colunas and not 'active' in status:
                elemento.click()
            if not coluna in colunas and 'active' in status:
                elemento.click()
        # fechando a tela das visualizações de colunas
        driver.find_element(By.TAG_NAME, 'body').send_keys(Keys.ESCAPE)
        time.sleep(3)
        # obtendo o campo da data de entrega
        campo_data = WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//*[@id="dataPrazoEntregaOSForm"]'))
            )
        # preenchendo o campo data de entrega
        campo_data.send_keys(retornar_periodo())
        time.sleep(3)
        # clicando em aplicar
        WebDriverWait(driver, 10).until(
            EC.element_to_be_clickable((By.XPATH, '//div[@class="drp-buttons"]//button[text()="Aplicar" and not(@disabled)]'))
            ).click()
        time.sleep(4)
    except Exception as e:
        raise RuntimeError(f'Erro ao aplicar filtros: {e}')

In [7]:
# função que obtém a lista das amostras
def obter_dados(driver):
    try:        
        # obtendo a lista de elementos
        lista_elementos = driver.find_elements(By.CSS_SELECTOR, '#tableOrdemdeServico tr.even, #tableOrdemdeServico tr.odd')

        if len(lista_elementos) < 1:
            print('Nenhum elemento foi encontrado')
            return []

        amostras = []
        for linha in lista_elementos:
            cliente = linha.find_elements(By.TAG_NAME, 'td')[4].text
            status_os = linha.find_elements(By.TAG_NAME, 'td')[1].text
            amostra = linha.find_elements(By.TAG_NAME, 'td')[2].text
            prioridade = linha.find_elements(By.TAG_NAME, 'td')[3].text
            amostras.append((status_os, amostra, cliente, prioridade))
            
        return amostras
    except Exception as e:
        raise RuntimeError(f'Erro ao obter os dados')

In [8]:
def sair_sistema(driver):
    try:
        driver.find_element(By.XPATH, '//a[@href="/sair"]').click()
        time.sleep(4)
        driver.quit()
        return True
    except Exception as e:
        raise RuntimeError(f'Erro ao sair do sistema utilizando o botão "sair"')
    finally:
        driver.quit()
        print('Navegador Fechado')

In [9]:
# Iniciar automação
def iniciar_automacao():
    driver = None
    try:
        # iniciando o driver
        driver = obter_driver()
        print('Driver Inicializado')

        # Realizando login
        logar(driver)
        print('Logado')

        # Aplicando as configurações
        aplicar_configuracoes(driver)
        print('Aplicado filtros e configurações')

        # Obtendo os dados
        dados = obter_dados(driver)
        print('Dados obtidos')

        return dados
    except Exception as e:
        print(f'Erro: {e}')
    finally:
        if driver:
            sair_sistema(driver)

In [10]:
# testes
dados = iniciar_automacao()
# driver = obter_driver()
# logado = logar(driver)
# aplicar_configuracoes(driver)
# dados = obter_dados(driver)
# sair_sistema(driver)

Driver Inicializado
Login realizado com sucesso!
Logado
Aplicado filtros e configurações
Dados obtidos
Navegador Fechado


In [16]:
for status_os, amostra, cliente, prioridade in dados:
    print(prioridade)

Padrão
Padrão
Padrão
Padrão
Padrão
Padrão
Padrão
Padrão
Padrão


In [27]:
# criar a função que envia e-mail
import smtplib
from email.mime.text import MIMEText
from dotenv import load_dotenv
from datetime import date
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText

load_dotenv()

def criar_corpo_str(dados: list):
    # Anexar uma versão de texto simples é uma boa prática
    plain_text_body = f"Prezada, Rayara!,\n\nSegue a listagem de amostras que deverão ser liberadas hoje::\n\n"
    if not dados:
        plain_text_body += "Nenhum dado encontrado para o período.\n"
    else:
        for status_os, amostra, cliente, prioridade in dados:
            plain_text_body += f"Status OS: {status_os}, Amostra: {amostra}, Cliente: {cliente}, Prioridade: {prioridade}\n"
    plain_text_body += "\nAtenciosamente,\nSua Automação"

    return plain_text_body

def criar_corpo_html(dados: list):
    html_content = '<p>Nenhum dado encontrado para o período.</p>'
    if not dados:
        return html_content
    html_content = """
<html>
    <head></head>
    <body>
        <p>Prezada, Rayara!</p>
        <p>Segue a listagem de amostras que deverão ser liberadas hoje:</p>
        <table style="width:100%; border-collapse: collapse;">
            <thead>
                <tr style="background-color: #f2f2f2;">
                    <th style="border: 1px solid black; padding: 8px; text-align: left;">Status OS</th>
                    <th style="border: 1px solid black; padding: 8px; text-align: left;">Amostra</th>
                    <th style="border: 1px solid black; padding: 8px; text-align: left;">Cliente</th>
                    <th style="border: 1px solid black; padding: 8px; text-align: left;">Prioridade</th>
                </tr>
            </thead>
            <tbody>
""" 
    for status_os, amostra, cliente, prioridade in dados:
        html_content += f"""
    <tr>
        <td style="border: 1px solid black; padding: 8px;">{status_os}</td>
        <td style="border: 1px solid black; padding: 8px;">{amostra}</td>
        <td style="border: 1px solid black; padding: 8px;">{cliente}</td>
        <td style="border: 1px solid black; padding: 8px;">{prioridade}</td>
    </tr>
 """
        
    html_content += """ 
    </tbody>
        </table>
        <p>Atenciosamente,<br>Sua Automação</p>
    </body>
</html>
"""
    return html_content

def enviar_email(dados: list):
    # Configurações do servidor SMTP
    servidor = os.getenv('SERVIDOR_ZOHO')
    porta = os.getenv('PORTA')
    usuario = os.getenv('USUARIO')
    senha = os.getenv('SENHA')

    # Destinatário, assunto e copor do e-mail
    # obtendo a data de hoje
    data_hoje = date.today()
    data_hoje_str = date.strftime(data_hoje, '%d-%m-%Y')
    remetente = usuario
    assunto = f'Liberações do dia {data_hoje_str}'
    # criando corpo html
    corpo_html =  criar_corpo_html(dados)
    corpo_str = criar_corpo_str(dados)

    # Cria a mensagem de e-mail
    msg = MIMEMultipart('alternative')
    msg['Subject'] = assunto
    msg['From'] = remetente
    msg['To'] = 'ti@grupoqualityambiental.com.br'

    msg.attach(MIMEText(corpo_html, 'html'))
    msg.attach(MIMEText(corpo_str, 'plain'))

    # Conecta ao servidor SMTP
    try:
        with smtplib.SMTP(servidor, porta) as server:
            server.starttls()
            server.login(usuario, senha) 
            server.send_message(msg)
            print('E-mail enviado com sucesso!')
    except Exception as e:
        print(f'Erro ao enviar e-mail: {e}')

In [29]:

enviar_email(dados)

E-mail enviado com sucesso!
